In [ ]:
#The problems below analyze the nominal, (rather than the TIPS,) Treasury issues. 
# Accordingly, you should filter out all rows with type TIPS Note or TIPS bond.

In [54]:
#file path to data file
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime
import seaborn
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator, FormatStrFormatter)

quotes = pd.read_excel("treasury_quotes_2025-11-28.xlsx",sheet_name='selected quotes').set_index('KYTREASNO')
#quotes = quotes[~quotes['type'].isin(['note', 'bond'])]


### 1.1 Create the cashflow matrix, C, where each rows is a Treasury issue, each column is a date, and each entry is the cash paid on a (normalized) face value of $100.

In [55]:
def find_coupon_dates(issue_date, maturity_date):
    #Returns semiannual coupon dates after issue_date up to maturity_date.
    # Convert to datetime if needed
    if isinstance(issue_date, str):
        issue_date = pd.to_datetime(issue_date)
    if isinstance(maturity_date, str):
        maturity_date = pd.to_datetime(maturity_date)

    semiannual_offset = pd.DateOffset(months=6)
    dates = []
    current_date = maturity_date

    while current_date > issue_date:
        dates.append(current_date)
        current_date -= semiannual_offset

    return sorted(dates)


In [56]:
CF = pd.DataFrame(index=quotes.index)  # bonds = rows

all_dates = []  # to collect all coupon dates

for i in quotes.index:
    issue_date = quotes.loc[i, 'issue date']
    maturity_date = quotes.loc[i, 'maturity date']
    coupon_dates = find_coupon_dates(issue_date, maturity_date)
    
    all_dates.extend(coupon_dates)  # collect all dates for column index

all_dates = sorted(list(set(all_dates)))  # unique sorted dates
CF = pd.DataFrame(0, index=quotes.index, columns=all_dates)

# Fill in coupons and principal
for i in quotes.index:
    issue_date = quotes.loc[i, 'issue date']
    maturity_date = quotes.loc[i, 'maturity date']
    coupon_rate = quotes.loc[i, 'cpn rate'] / 2  # semiannual coupon per 100 face

    coupon_dates = find_coupon_dates(issue_date, maturity_date)
    
    for d in coupon_dates:
        CF.loc[i, d] = coupon_rate
    
    # Add principal at maturity
    CF.loc[i, maturity_date] += 100


/var/folders/rl/1p19jmyj1k357sd_cr_qw2yh0000gn/T/ipykernel_8373/3404277675.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.4375' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  CF.loc[i, d] = coupon_rate
/var/folders/rl/1p19jmyj1k357sd_cr_qw2yh0000gn/T/ipykernel_8373/3404277675.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.4375' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  CF.loc[i, d] = coupon_rate
/var/folders/rl/1p19jmyj1k357sd_cr_qw2yh0000gn/T/ipykernel_8373/3404277675.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.4375' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  CF.loc[i, d] = coupon_

In [57]:
CF = CF.resample('M', axis=1).sum()  # sum by month
CF = CF.loc[:, (CF != 0).any()]      # drop all-zero columns
CF

/var/folders/rl/1p19jmyj1k357sd_cr_qw2yh0000gn/T/ipykernel_8373/3079749178.py:1: FutureWarning: DataFrame.resample with axis=1 is deprecated. Do `frame.T.resample(...)` without axis instead.
  CF = CF.resample('M', axis=1).sum()  # sum by month
/var/folders/rl/1p19jmyj1k357sd_cr_qw2yh0000gn/T/ipykernel_8373/3079749178.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  CF = CF.resample('M', axis=1).sum()  # sum by month


,2023-11-30,2024-05-31,2024-11-30,2025-05-31,2025-11-30,2026-05-31,2026-11-30,2027-05-31,2027-11-30,2028-05-31,...,2031-05-31,2031-11-30,2032-05-31,2032-11-30,2033-05-31,2033-11-30,2034-05-31,2034-11-30,2035-05-31,2035-11-30
KYTREASNO,,,,,,,,,,,,,,,,,,,,,
208227,0.0000,0.0000,2.4375,2.4375,2.4375,102.4375,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0
208316,0.0000,0.0000,0.0000,2.1250,2.1250,2.1250,102.1250,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0
208399,0.0000,0.0000,0.0000,0.0000,1.9375,1.9375,1.9375,101.9375,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0
208484,0.0000,0.0000,0.0000,0.0000,0.0000,1.6875,1.6875,1.6875,101.6875,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0
208058,1.8125,1.8125,1.8125,1.8125,1.8125,1.8125,1.8125,1.8125,1.8125,101.8125,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0
208145,0.0000,2.1875,2.1875,2.1875,2.1875,2.1875,2.1875,2.1875,2.1875,2.1875,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0
208229,0.0000,0.0000,2.2500,2.2500,2.2500,2.2500,2.2500,2.2500,2.2500,2.2500,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0
208318,0.0000,0.0000,0.0000,2.0625,2.0625,2.0625,2.0625,2.0625,2.0625,2.0625,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0
208401,0.0000,0.0000,0.0000,0.0000,2.0000,2.0000,2.0000,2.0000,2.0000,2.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.0


### 1.2 Estimate the spot discount factors that rationalize the market prices of the treasuries. Do this via solving the linear system: p = Cz

### 1.3 Plot the time-to-maturity on the horizontal axis. On the vertical axis, plot the… 
#### - discount factors
### Make another plot, this time with the vertical axis displaying the
#### - semiannually compounded discount rates
#### - continuously compounded discount rates

